# Chapter02 텍스트 마이닝 첫걸음

- 이장의 핵심개념
    - 웹 크롤링으로 데이터를 수집한다
    - 키워드 추출의 방법을 알아본다.
    - 키워드 간의 연관 관계를 분석한다.
    - 텍스트 분석 결과를 시각화 한다.
    
### 2.1 웹 크롤링으로 기초 데이터 수집하기

이번 절에서는 '나무위키 최근 변경 페이지'의 텍스트 데이터르 웹 크롤링 Web Crawling 으로 수집한 다음, 데이터 내에서 등장한 키워드의 출현 빈도를 분석해 보겠다. 이를 통해 우리는 나무위키 페이지에서 현재 가장 '핫한'키워드가 무엇인지 분석할 수 있다. 웹 크롤링 혹은 웹 스크래핑 Web Scraping은 인터넷에 있는 웹페이지를 방문해서 페이지의 자료를 자동으로 수집하는 작업을 의미한다. 여기에선 파이썬으로 웹 크롤링을 진행하겠다.

#### 대상 페이지의 구조 살펴보기

크롤링을 위한 첫 번째 단계는 인터넷 익스플로러, 크롬 등의 웹브라우저를 실행하여 크롤링의 대상이 될 페이지 구조를 살펴보는 것이다. 
- 먼저 웹 브라우저의 '개발자 도구'를 실행한다. 단축키 Ctrl + Shift + l
- 리스트의 URL 정보를 수집

#### 웹 크롤링 라이브러리 사용하기

파이썬에서는 BeautifulSoup 과 requests라는 라이브러리로 웹 크롤러를 만들 수 있다. requests는 특정 URL로부터 HTML 문서를 가져오는 작업을 수행하고, BeautifulSoup 모듈은 HTML 문서에서 데이터를 추출하는 작업을 수행한다. 이 모듈들을 사용하기에 앞서, 터미널(cmd) 혹은 아나콘다 프롬프트를 실행하여 아래의 세 가지 파이썬 모듈을 설치해야한다.


In [1]:
# !pip3 install lxml beautifulsoup4 requests

  Using cached https://files.pythonhosted.org/packages/1f/1d/a4485412268b38043a6c0f873245b5d9315c6615bcf44776759a2605dca5/lxml-4.6.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d1/41/e6495bd7d3781cee623ce23ea6ac73282a373088fcd0ddc809a047b18eae/beautifulsoup4-4.9.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/29/c1/24814557f1d22c56d50280771a17307e6bf87b70727d975fd6b2ce6b014a/requests-2.25.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/36/69/d82d04022f02733bf9a72bc3b96332d360c0c5307096d76f6bb7489f7e57/soupsieve-2.2.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/19/c7/fa589626997dd07bd87d9269342ccb74b1720384a4d739a1872bd84fbe68/chardet-4.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/09/c6/d3e3abe5b4f4f16cf0dfc9240ab7ce10c2baa0e268989a4e3ec19e90c84e/urllib3-1.26.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org

In [7]:
# -*- coding: utf-8 -*-

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

다음 코드에서는 requests.get()함수로 URL의 HTML 문서를 가져온 뒤, 이를 BeautifulSOup()클래스의 soup 객체로 변환한다. 그리고 find(), find_all() 함수를 사용하여 특정 HTML 태그 혹은 특정 HTML 클래스를 가진 데이터를 가져온다.

- 페이지 리스트 가져오기

In [14]:
import requests
from bs4 import BeautifulSoup
import re

# 크롤링할 사이트 주소를 정의한다
source_url = "https://namu.wiki/RecentChanges"

# 사이트의 HTML 구조에 기반하여 크롤링을 수행한다.
req = requests.get(source_url)
html = req.content
soup = BeautifulSoup(html, 'lxml')
contents_table = soup.find(name = "table", attrs = {"class":"table-hover"})
table_body = contents_table.find(name="tbody")
table_rows = table_body.find_all(name="tr")

# a 태그의 href 속성을 리스트로 추출하여 크롤링할 페이지 리스트를 생성한다.
page_url_base = "https://namu.wiki"
page_urls = []
for index in range(0, len(table_rows)):
    first_td = table_rows[index].find_all('td')[0]
    td_url = first_td.find.all('a')
    if len(td_url) > 0:
        page_url = page_url_base + td_url[0].get('href')
        page_urls.append(page_url)
        
# 중복 url을 제거한다.
page_urls = list(set(page_urls))
for page in page_urls[:5]:
    print(page)

AttributeError: 'NoneType' object has no attribute 'find'

In [12]:
import requests
from bs4 import BeautifulSoup
import re

# 크롤링할 사이트 주소를 정의합니다.
source_url = "https://namu.wiki/RecentChanges"

# 사이트의 html 구조에 기반하여 크롤링을 수행합니다.
req = requests.get(source_url)
html = req.content
soup = BeautifulSoup(html, 'lxml')
contents_table = soup.find(name="table")
table_body = contents_table.find(name="tbody")
table_rows = table_body.find_all(name="tr")

AttributeError: 'NoneType' object has no attribute 'find'